<a href="https://colab.research.google.com/github/verma-priyanka/EarthEngine/blob/master/01_Colab_EEPythonAPI_Intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p><img alt="Colaboratory logo" height="45px" src="/img/colab_favicon.ico" align="left" hspace="10px" vspace="0px"></p>

<h1> Google Colaboratory |  Earth Engine Python API </h1>

Colaboratory is a free Jupyter notebook environment that requires no setup and runs entirely in the cloud.

----

## Cells
A notebook is a list of cells. Cells contain either explanatory text or executable code and its output.

#### This is text cell. 
> Text cells support Markdown Language


In [0]:
# This is a code cell 
hello_world = 'Hello World'
print(hello_world)

Hello World


-----
# Kernels 
Every notebook runs on a kernel. When you run a code cell, that code is executed within the kernel and any output is returned back to the cell to be displayed. 

Colab supports Python 2.7 and Python 3.6.

In [0]:
# Declare variable hello_world
hello_world = 'Hello World'

In [0]:
# Print variable hello_world
print(hello_world)

Hello World




---


# Saving Notebooks
Notebooks are saved in Google Drive with an .ipynb extension. 

## GitHub
*Saving a Colab notebook to GitHub requires giving Colab permission to push the commit to your repository.*


To save your notebook to GitHub:
1.   Choose File→Save a copy to GitHub.  
2.   Select Repository
3.   Provide file name under file path
4.   Hit OK



---




# HTML
Colab supports HTML

Adding '%%html' at the beginning of a code cell reads the cell as HTML instead of executing it as python code



In [0]:
%%html
<text style='color: Red;'><b>HTML Text!</b></text>

In [0]:
# This results in an error in python
<text style='color: Red;'><b>HTML Text!</b></text>



---


# Stack Overflow
If your code results in an error, Colab provides an quick way to search errors on StackOverflow.com through the Google search engine.

In [0]:
print(hello_world + 1)

Hello World1


<br>
<br>
<br>


---
# Google Earth Engine: Python API
The Earth Engine Python API facilitates interacting with Earth Engine servers using the Python programming language.

<br>

## Installing a Library
GEE's Python API is accessed through the **earthengine-api** library. 
<br>
Python libraries can be easily downloaded in colab using **pip**, an python package installer

In [0]:
# Download library using pip
!pip install earthengine-api

In [0]:
# Import the earthengine-api library
# The Python API package is called ee- must be imported into every new Python session and script.
import ee

## Authentication
Authenticating is necessary each time you begin working the GEE API in a Colab notebook.

1.   Execute the code in the cell below and open the outputted URL
2.   Choose your account with access to Google Earth Engine 
3.   Allow the Python Authenticator to access your account
4.   Copy the code and enter it as the verification code below
5.   Hit Enter
6.   You should see 'Successfully saved authorization token.' if the authentication was successful




In [0]:
ee.Authenticate()

In [0]:
# Initialize the library
# The library must be initialized for every new session and script
ee.Initialize()

## Test the API

The code below prints the elevation of Mount Everest using the [Shuttle Radar Topography Mission (SRTM) Digital Elevation](https://developers.google.com/earth-engine/datasets/catalog/USGS_SRTMGL1_003) data. 

According to the data catalog shows that the image was collected for a 10-day time period from *Feb 11th to Feb 22nd 2000*.

Some additional information can be found under the Bands tab like the data has **one band called elevation** with a **resolution of 30m**.



In [0]:
# Specify path to image from the Earth Engine Data Catalog
# This can be found under Earth Engine Snippet on the Data Catalog Page
dem = ee.Image('USGS/SRTMGL1_003')

# Generate a vector point using the ee.Geometry.Point Function 
# Pass longitude(x), latitude(y) values in a list as parameters
# It is important to retain the order: x,y
xy = ee.Geometry.Point([-71.815830, 42.250420])

# Sample the pixels of image at the specified location
# Select the first sample
# The getInfo() function extracts the elevation information
elev = dem.sample(xy).first().get('elevation').getInfo()

# Print elevation information
print('Clark University Elevation (m):', elev)

Clark University elevation (m): 154


# Exercise 1
Edit the code below to print the elevation information for **Mount Everest**


[Use this website to get the lat/lon information](https://www.latlong.net/convert-address-to-lat-long.html )

In [0]:
# Specify path to image from the Earth Engine Data Catalog
dem = ee.Image('USGS/SRTMGL1_003')

# Pass longitude(x), latitude(y) values in a list as parameters
# Lat/Lon for Clark University specified
xy = ee.Geometry.Point([-71.815830, 42.250420])

# Sample the pixels of image at the specified location
# Select the first sample
# The getInfo() function extracts the elevation information
elev = dem.sample(xy).first().get('elevation').getInfo()

# Print elevation information
print('Mount Everest Elevation (m):', elev)



---

# Mapping

The **folium** package can be used to display earth engine images on an interactive map

In [0]:
# Download the folium library
!pip install folium

In [0]:
# Import Library
import folium

In [0]:
# Define a function to display Earth Engine image tiles to folium map.
# The function takes:
#   1. Earth Engine Image
#   2. Visual parameters as dictionary
#   3. A Name as a string

def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)



---


<br>



# Image Collection
An image collection refers to a set of Earth Engine images. For example, the collection of all Landsat 8 images is an ImageCollection. Like the SRTM image, image collections also have an ID.

Example: [Landsat 8 Image Collection](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_SR)


A collection represents every scene collected at every location and time period. It is useful to extract a single image or a subset of images. The way to limit the collection by time or space is by filtering it. 

In [0]:
# Location to extract imagery for using geographic coordinates
point = ee.Geometry.Point([-71.815830, 42.250420])
# Dates
startDate = '2016-01-01'
endDate = '2017-01-31'

# imageCollection path
image = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")

# Filter by geom point
image = image.filterBounds(point)

# Specify time period of interest
image = image.filterDate(startDate, endDate)

image = image.min()

# ----TRUE COLOR COMPOSITE------
visParams = {'bands': ['B4', 'B3', 'B2'],
                'min': 0,
                'max': 3000,
                'gamma': 1.4}

Set up map by providing default location, zoom level and size.

Add layer to map

In [0]:
# Create a folium map object 
# Folium defaults to openstreetmap basemap if a tile isn't specified
# Format for location coordinates: Latitude, Longitude
# Height for 
imageryMap = folium.Map(location=[42.250420, -71.815830], zoom_start=7, height=500)

# Create a folium map object with the add_ee_layer
folium.Map.add_ee_layer = add_ee_layer

# Add the layer to the map using the defined function
imageryMap.add_ee_layer(image, visParams, 'True Color Composite')

# Add a layer control panel to the map
# Allows you to turn layers on and off
imageryMap.add_child(folium.LayerControl())

# Display the map
display(imageryMap)



---



# Exercise 2: Create a False Color Composite

Create a false-color composite for the same geographic area. Use the same visual parameters as the true color composite. 

Edit the code below and update the following:
*   startDate = *2017-01-01*
*   endDate = *2018-01-31*
*   Bands: *B5, B4, B3*



(Hint: You are updating 3 lines of code)




In [0]:
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)


# Location to extract imagery for using geographic coordinates
point = ee.Geometry.Point([-71.815830, 42.250420])

# Dates
startDate = '2017-01-01'
endDate = '2018-01-31'

# imageCollection path
image = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")

# Filter by geom point
image = image.filterBounds(point)

# Specify time period of interest
image = image.filterDate(startDate, endDate)

image = image.min()

# ----TRUE COLOR COMPOSITE------
visParams = {'bands': ['B5', 'B4', 'B3'],
                'min': 0,
                'max': 3000,
                'gamma': 1.4}

# Create a folium map object 
# Folium defaults to openstreetmap basemap if a tile isn't specified
# Format for location coordinates: Latitude, Longitude
# Height for 
imageryMap = folium.Map(location=[42.250420, -71.815830], zoom_start=7, height=800)

# Create a folium map object with the add_ee_layer
folium.Map.add_ee_layer = add_ee_layer

# Add the layer to the map using the defined function
imageryMap.add_ee_layer(image, visParams, 'True Color Composite')

# Add a layer control panel to the map
# Allows you to turn layers on and off
imageryMap.add_child(folium.LayerControl())

# Display the map
display(imageryMap)